In [2]:
import torch
import torch.nn as nn
import pandas as pd 
from torchtext.vocab import vocab
from collections import Counter
from sklearn.metrics import f1_score
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from sklearn.metrics import  confusion_matrix
import numpy as np

In [3]:
male_names = pd.read_csv(r'https://gist.githubusercontent.com/mbejda/7f86ca901fe41bc14a63/raw/38adb475c14a3f44df9999c1541f3a72f472b30d/Indian-Male-Names.csv')
female_names = pd.read_csv(r'https://gist.githubusercontent.com/mbejda/9b93c7545c9dd93060bd/raw/b582593330765df3ccaae6f641f8cddc16f1e879/Indian-Female-Names.csv')
names_df = male_names.append(female_names)
names_df.to_csv("indian_names.csv", index= False )
names_df = pd.read_csv("indian_names.csv")

In [4]:
names_np = names_df.to_numpy()

In [7]:
import math
#remove nan values
def remove_nan_entries(names_np):
  nan_entry = []
  for i in range(len(names_np)):
    try:
        if(math.isnan(names_np[i][0])):
            nan_entry.append(i)
    except:
        pass
  if(nan_entry != []):
    names_np=np.delete(names_np,np.array(nan_entry),axis=0)
  return names_np

#remove nan values
def clip_names_with_length_more_than_th(names_np,th=20):
  entry =[]
  for i in range(len(names_np)):
    try:
        if(len(names_np[i][0])>th):
            entry.append(i)
    except:
        pass
  if(entry != []):
    names_np=np.delete(names_np,np.array(entry),axis=0)
  return names_np

#remove nan values
def zero_padding(m_name,th=20):
  return (th-len(m_name))//2*'0'+ m_name+(th-len(m_name))//2*'0'+len(m_name)%2*'0'

#remove nan values
def padding_to_make_consant_length(names_np):
  for i in range(len(names_np)):
    names_np[i][0] = zero_padding(names_np[i][0])
  return names_np

names_np = remove_nan_entries(names_np)
names_np = clip_names_with_length_more_than_th(names_np)
names_np = padding_to_make_consant_length(names_np)

[]


In [9]:
# Padding to make constant length
for i in range(len(names_np)):
    try:
        if(len(names_np[i][0])<20):
            names_np[i][0]=(20-len(names_np[i][0]))//2*'0'+names_np[i][0]+(20-len(names_np[i][0]))//2*'0'+len(names_np[i][0])%2*'0'
    except:
        pass
x=names_np[:,0]
y=names_np[:,1:2]

In [10]:
jn = ""
for entry in x:
  jn = jn + str(entry)

In [11]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
lbl=LabelEncoder()
one=OneHotEncoder()

y=lbl.fit_transform(y)
y=one.fit_transform(y.reshape(len(y),1)).toarray()

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
create_own_mapping = False
if(create_own_mapping == True):
  chars = sorted(list(set(jn)))
  mapping = dict((c, i) for i, c in enumerate(chars))
else:
  mapping = {'\t': 0, ' ': 1, '$': 2, '&': 3, '(': 4, ')': 5, ',': 6, '-': 7, '.': 8, '/': 9, '0': 10, '1': 11, '2': 12, '4': 13, '8': 14, '@': 15, '[': 16, '`': 17, 'a': 18, 'b': 19, 'c': 20, 'd': 21, 'e': 22, 'f': 23, 'g': 24, 'h': 25, 'i': 26, 'j': 27, 'k': 28, 'l': 29, 'm': 30, 'n': 31, 'o': 32, 'p': 33, 'q': 34, 'r': 35, 's': 36, 't': 37, 'u': 38, 'v': 39, 'w': 40, 'x': 41, 'y': 42, 'z': 43, 'ì': 44, 'ं': 45, 'अ': 46, 'आ': 47, 'इ': 48, 'ई': 49, 'उ': 50, 'ऐ': 51, 'क': 52, 'ख': 53, 'ग': 54, 'च': 55, 'छ': 56, 'ज': 57, 'ट': 58, 'ण': 59, 'त': 60, 'द': 61, 'ध': 62, 'न': 63, 'प': 64, 'फ': 65, 'ब': 66, 'भ': 67, 'म': 68, 'य': 69, 'र': 70, 'ल': 71, 'व': 72, 'श': 73, 'ष': 74, 'स': 75, 'ह': 76, '़': 77, 'ा': 78, 'ि': 79, 'ी': 80, 'ु': 81, 'ू': 82, 'े': 83, 'ै': 84, 'ो': 85, '्': 86, '\u200d': 87, '�': 88}

In [13]:
names = []
for line in x:
  encoded_seq = []
  for char in line:
    encoded_seq.append(int(mapping[char]))
  names.append(encoded_seq)

In [14]:
x=np.array(names).reshape(len(names),20,1)

In [24]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=42)

In [22]:
#Model
from keras.layers import Input, Dense,LSTM
from keras.models import Model
inp = Input(shape=(20,1))
lstm = LSTM(30,return_sequences=True)(inp)
lstm = LSTM(70,return_sequences=True)(lstm)
lstm = LSTM(30)(lstm)
out = Dense(2,activation='softmax')(lstm)

model = Model(inp,out)
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 20, 1)]           0         
                                                                 
 lstm_3 (LSTM)               (None, 20, 30)            3840      
                                                                 
 lstm_4 (LSTM)               (None, 20, 70)            28280     
                                                                 
 lstm_5 (LSTM)               (None, 30)                12120     
                                                                 
 dense_1 (Dense)             (None, 2)                 62        
                                                                 
Total params: 44,302
Trainable params: 44,302
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
from keras.callbacks import ModelCheckpoint
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=["accuracy"])
checkpoint = ModelCheckpoint('best_weights.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
model.fit(x_train,y_train,batch_size=50,epochs=100,verbose=1,callbacks=[checkpoint],validation_data=(x_test,y_test))

Epoch 1/100
414/414 [==============================] - ETA: 0s - loss: 0.5562 - accuracy: 0.7103

414/414 [==============================] - 23s 43ms/step - loss: 0.5562 - accuracy: 0.7103 - val_loss: 0.5378 - val_accuracy: 0.7179
Epoch 2/100
413/414 [============================>.] - ETA: 0s - loss: 0.5402 - accuracy: 0.7135

414/414 [==============================] - 17s 40ms/step - loss: 0.5402 - accuracy: 0.7134 - val_loss: 0.5294 - val_accuracy: 0.7263
Epoch 3/100
413/414 [============================>.] - ETA: 0s - loss: 0.5305 - accuracy: 0.7188

414/414 [==============================] - 17s 42ms/step - loss: 0.5305 - accuracy: 0.7186 - val_loss: 0.5176 - val_accuracy: 0.7349
Epoch 4/100
414/414 [==============================] - ETA: 0s - loss: 0.5240 - accuracy: 0.7199

414/414 [==============================] - 17s 41ms/step - loss: 0.5240 - accuracy: 0.7199 - val_loss: 0.5216 - val_accuracy: 0.7196
Epoch 5/100
413/414 [============================>.] - ETA: 0s - loss: 0.5158 - accuracy: 0.7260

...

414/414 [==============================] - 17s 41ms/step - loss: 0.1069 - accuracy: 0.9619 - val_loss: 0.3608 - val_accuracy: 0.9099
Epoch 97/100
413/414 [============================>.] - ETA: 0s - loss: 0.1163 - accuracy: 0.9587

414/414 [==============================] - 17s 41ms/step - loss: 0.1163 - accuracy: 0.9587 - val_loss: 0.3723 - val_accuracy: 0.8974
Epoch 98/100
413/414 [============================>.] - ETA: 0s - loss: 0.1050 - accuracy: 0.9633

414/414 [==============================] - 17s 41ms/step - loss: 0.1049 - accuracy: 0.9633 - val_loss: 0.3512 - val_accuracy: 0.9057
Epoch 99/100
413/414 [============================>.] - ETA: 0s - loss: 0.0950 - accuracy: 0.9663

414/414 [==============================] - 17s 41ms/step - loss: 0.0950 - accuracy: 0.9664 - val_loss: 0.3523 - val_accuracy: 0.9089
Epoch 100/100
414/414 [==============================] - ETA: 0s - loss: 0.0994 - accuracy: 0.9650

414/414 [==============================] - 19s 46ms/step - loss: 0.0994 - accuracy: 0.9650 - val_loss: 0.3686 - val_accuracy: 0.9039


In [ ]:
import matplotlib.pyplot as plt
print(model.history.history.keys())
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_loss'])
plt.plot(model.history.history['val_accuracy'])
plt.legend()
plt.show()

In [37]:
model.evaluate(x_test,y_test)

277/277 [==============================] - 5s 17ms/step - loss: 0.3686 - accuracy: 0.9039


[0.36855560541152954, 0.9039069414138794]

In [108]:
def encoded_sequence(padded_str,mapping):
  encoded_seq = []
  for char in padded_str:
    encoded_seq.append(int(mapping[char]))
  return encoded_seq

def predict_gender(model,m_name):
  m_name = m_name.lower()
  encoded_seq = encoded_sequence(zero_padding(m_name),mapping)
  np_encoded_seq = np.array(encoded_seq).reshape(1,20,1)
  confidence = model.predict(np_encoded_seq)[0]
  confidence = list(confidence)
  if(confidence.index(max(confidence)) == 0):
    return m_name.upper() + " -> Female " + str(round(max(confidence)*100)) + "%"
  else:
    return m_name.upper() + " -> Male " + str(round(max(confidence)*100)) + "%"

In [127]:
names = ["PK Adhya Kumari","Aarvi Saxena KK","Ahaana","Bela Caio","Dhara","Diya Mirza","Prisha Upadhyay","Aanya Pandey","Hari Mehta","Hans Hanthi","Jagan Mohan","Jai Shree","Kala Bagga","Kamal Hasan","Kanan Gill","Milan","Rahul Kumar","Tarak","Aatma Ram","Vihaan Jain","Wali Rehmani"]
for name in names:
  print(predict_gender(model,name))

PK ADHYA KUMARI -> Female 85%
AARVI SAXENA KK -> Female 100%
AHAANA -> Female 97%
BELA CAIO -> Female 100%
DHARA -> Female 97%
DIYA MIRZA -> Female 100%
PRISHA UPADHYAY -> Female 90%
AANYA PANDEY -> Female 96%
HARI MEHTA -> Female 96%
HANS HANTHI -> Male 77%
JAGAN MOHAN -> Male 100%
JAI SHREE -> Male 69%
KALA BAGGA -> Female 57%
KAMAL HASAN -> Male 100%
KANAN GILL -> Male 100%
MILAN -> Male 95%
RAHUL KUMAR -> Male 100%
TARAK -> Female 81%
AATMA RAM -> Male 100%
VIHAAN JAIN -> Male 99%
WALI REHMANI -> Male 76%
